In [1]:


%%capture
%load_ext autoreload
%autoreload 2


import numpy as np
import pandas as pd
import os
import sys
from dotenv import load_dotenv
from pathlib import Path
import datetime as d 
load_dotenv()

# get the current directory
current_dir = os.getcwd()

# change the current working directory to the parent directory
os.chdir(os.path.dirname(current_dir))

path = os.getcwd()
parent = os.path.abspath(os.path.join(path, os.pardir))

USRPATH = "/Users/ducjeremyvu/dev/"


sys.path.append(Path().absolute().parent)
sys.path.append("/Users/ducjeremyvu/dev/wspace/nb")
sys.path.append(parent)
sys.path.append(...)

os.getcwd()


# from g_api import gservice


In [2]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

def get_last_three_month_period():
    today = datetime.today()
    
    # End date is the last day of the previous month
    end_date = today.replace(day=1) - timedelta(days=1)
    
    # Start date is the first day of the month three months ago
    start_date = today.replace(day=1) - relativedelta(months=3)
    
    # Format the dates in 'YYYY-MM-DD'
    return start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')

# Example usage
start, end = get_last_three_month_period()
print(f"Start: {start}, End: {end}")


Start: 2024-08-01, End: 2024-10-31


In [3]:
from ucmdata.database.psql import Postgres
from sshtunnel import SSHTunnelForwarder

server = SSHTunnelForwarder(("djvsrv2.netbird.cloud", 22),
         ssh_username="djv",
         ssh_password="ucmsysadmin",
         remote_bind_address=('ucm-main-read-replica-public.c03qb56spoj7.eu-central-1.rds.amazonaws.com', 5432),
         local_bind_address=('localhost', 60000))
        
server.start()



INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.9p1)
INFO:paramiko.transport:Authentication (publickey) failed.
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.9p1)
INFO:paramiko.transport:Authentication (publickey) failed.
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.9p1)
INFO:paramiko.transport:Authentication (password) successful!


In [4]:

p = Postgres(port=60000, host="localhost")

query = f"""
WITH merged AS (
    SELECT
        *,
        CASE WHEN p.company_id in(37568,
            37379,
            46368,
            49813,
            37853,
            51318,
            46395,
            49808,
            49692) THEN
            'airport'
        ELSE
            dt.code
        END AS team_name
    FROM
        user_job_dates AS ujd
        JOIN job_applications AS ja ON ja.id = ujd.job_application_id
        JOIN users AS u ON u.id = ja.user_id
        JOIN jobs AS j ON j.id = ja.job_id
        JOIN projects AS p ON p.id = j.project_id
        JOIN department_teams AS dt ON dt.id = p.team_id
    WHERE
        ujd.checkin_time >= '{start}'
        AND ujd.checkin_time <= '{end}'
),
ranked AS (
    SELECT
        user_id,
        SEQUENCE,
        team_name,
        COUNT(*) AS checkins,
        ROW_NUMBER() OVER (PARTITION BY user_id,
            SEQUENCE ORDER BY user_id,
            SEQUENCE) AS rn
    FROM
        merged
    GROUP BY
        user_id,
        SEQUENCE,
        team_name ORDER BY
            user_id,
            SEQUENCE,
            team_name,
            checkins DESC
)
SELECT
    user_id,
    SEQUENCE,
    team_name,
    CASE WHEN team_name = 'fairs_events' THEN
        1200
    WHEN team_name = 'gastronomy' THEN
        1400
    WHEN team_name = 'hands_service' THEN
        1300
    WHEN team_name = 'promotions' THEN
        1500
    WHEN team_name = 'airport' THEN
        1100
    ELSE
        0
    END AS KOST
FROM
    ranked
WHERE
    rn = 1 ORDER BY
        user_id, SEQUENCE, team_name;

"""
shift_data = p.get_data(query=query).dropna()
shift_data["kost"] = shift_data["kost"].astype(int)
shift_data["sequence"] = shift_data["sequence"].astype(int)

  self._data = read_sql_query(query, conn, params=params)

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shift_data["kost"] = shift_data["kost"].astype(int)

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shift_data["sequence"] = shift_data["sequence"].astype(int)



In [5]:
server.stop()

In [6]:
shift_data

,user_id,sequence,team_name,kost
0,6568,1,fairs_events,1200
1,7703,133,fairs_events,1200
2,8015,23,fairs_events,1200
3,8015,24,fairs_events,1200
4,8015,25,fairs_events,1200
...,...,...,...,...
3378,88164,1,promotions,1500
3379,88213,1,promotions,1500
3380,88222,1,promotions,1500
3381,88734,1,promotions,1500


In [7]:

def add_prefix(num):
    num_str = str(num)
    length = len(num_str)
    
    if length == 5:
        return "A" + num_str
    elif length == 4:
        return "A0" + num_str
    elif length == 3:
        return "A00" + num_str
    elif length == 2:
        return "A000" + num_str
    else:
        return "A0000" + num_str



shift_data["PNR"] = shift_data["user_id"].apply(lambda x: add_prefix(x))

shift_data

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shift_data["PNR"] = shift_data["user_id"].apply(lambda x: add_prefix(x))



,user_id,sequence,team_name,kost,PNR
0,6568,1,fairs_events,1200,A06568
1,7703,133,fairs_events,1200,A07703
2,8015,23,fairs_events,1200,A08015
3,8015,24,fairs_events,1200,A08015
4,8015,25,fairs_events,1200,A08015
...,...,...,...,...,...
3378,88164,1,promotions,1500,A88164
3379,88213,1,promotions,1500,A88213
3380,88222,1,promotions,1500,A88222
3381,88734,1,promotions,1500,A88734


In [8]:
shift_data[(shift_data["PNR"] == "A10244")&(shift_data["sequence"] == 58)]

,user_id,sequence,team_name,kost,PNR


In [9]:
def assign_kost(x):
    row = shift_data[(shift_data["PNR"] == x.PNR) & (shift_data["sequence"] == x.VERTNR)]
    
    if row.empty:
        return 1100
    else:
        # Use .iloc to access the first row if it exists
        return row.iloc[0]["kost"]

In [10]:
import re
folder_path = "/Users/ducjeremyvu/Downloads/ucmdownload/PR_202410_131"

# Compile the regex pattern to find the substring between numbers_underscore and .csv
pattern = re.compile(r'\d+_\d+_(.*?)\.csv')


# Dictionary comprehension to extract the desired part from filename
folder_extract = {pattern.search(file.name).group(1): file 
                  for file in Path(folder_path).glob("*.csv") 
                  if pattern.search(file.name)}





# Dictionary comprehension to read CSV files into pandas DataFrames
csv_read = {key: pd.read_csv(path, sep=";", dtype=str)
            for key, path in folder_extract.items()}


In [11]:
csv_read.keys()

dict_keys(['danger', 'wd', 'pers_edit', 'PSTKST', 'sv', 'steuer', 'bank', 'pers', 'contract'])

In [12]:
data_wd = csv_read.get("wd")
data_wd

data_wd["VERTNR"] = data_wd["VERTNR"].astype(int)

In [13]:

data_wd["KST"] = data_wd.apply(assign_kost, axis=1)

data_wd["KST"].unique()

array([1200, 1500, 1100, 1400, 1300])

In [14]:
data_wd.to_csv(str(folder_extract.get("wd")).split(".")[0]+"_edit.csv", index=False, sep=";")

# Template
